In [ ]:
! pip install openai==0.28.1
! pip install python-dotenv

In [1]:
import os

# Load environment variables from .env file
from dotenv import load_dotenv

load_dotenv()


### Work with openai==0.28.1
#### Remember to use GPT-4 instead of GPT-3.5-turbo, or you will encounter an error that `operation not permitted`

In [26]:
import os
import openai
import requests

openai.api_base = os.environ.get("AZURE_OPENAI_BASE")
openai.api_version = "2023-08-01-preview"
openai.api_type = 'azure'
openai.api_key = os.environ.get("OPENAI_API_KEY")

def setup_byod(deployment_id: str) -> None:
    """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.

    :param deployment_id: The deployment ID for the model to use with your own data.

    To remove this configuration, simply set openai.requestssession to None.
    """

    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):

     def send(self, request, **kwargs):
         request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
         return super().send(request, **kwargs)

    session = requests.Session()

    # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
    session.mount(
        prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
        adapter=BringYourOwnDataAdapter()
    )

    openai.requestssession = session

aoai_deployment_id = os.environ.get("MODEL_TYPE")
setup_byod(aoai_deployment_id)

completion = openai.ChatCompletion.create(
    messages=[{"role": "user", "content": "What are the differences between Azure Machine Learning and Azure AI services?"}],
    model="gpt-4",
    engine="gpt-4",
    deployment_id="gpt-4",
    dataSources=[
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": os.environ.get("AZURE_SEARCH_ENDPOINT"),
                "key": os.environ.get("AZURE_SEARCH_KEY"),
                "indexName": os.environ.get("AZURE_SEARCH_INDEX"),
            }
        }
    ]
)
print(completion)

{
  "id": "609f0b14-fb04-4029-b33c-345956ee7978",
  "model": "gpt-4",
  "created": 1701772777,
  "object": "extensions.chat.completion",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "The requested information is not found in the retrieved data. Please try another query or topic.",
        "end_turn": true,
        "context": {
          "messages": [
            {
              "role": "tool",
              "content": "{\"citations\": [], \"intent\": \"[\\\"Differences between Azure Machine Learning and Azure AI services\\\"]\"}",
              "end_turn": false
            }
          ]
        }
      }
    }
  ],
  "usage": {
    "prompt_tokens": 1345,
    "completion_tokens": 12,
    "total_tokens": 1357
  }
}
